In [449]:
import pandas as pd
import numpy as np
from dash import Dash, html, dcc, Input, Output, dash_table, ctx
import dash_bootstrap_components as dbc
import plotly.express as px
import altair as alt
import json
from vega_datasets import data
from fuzzywuzzy import fuzz, process

df = pd.read_csv("HIV.csv", low_memory=False)

In [315]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40540 entries, 0 to 40539
Data columns (total 22 columns):
 #   Column                                                                          Non-Null Count  Dtype  
---  ------                                                                          --------------  -----  
 0   DATAFLOW                                                                        40540 non-null  object 
 1   REF_AREA:Geographic area                                                        40540 non-null  object 
 2   INDICATOR:Indicator                                                             40540 non-null  object 
 3   SEX:Sex                                                                         40540 non-null  object 
 4   TIME_PERIOD:Time period                                                         40540 non-null  object 
 5   OBS_VALUE:Observation Value                                                     40540 non-null  object 
 6   UNIT_MULTIPLIE

Pivot the dataframe to have each indicator on seperate column and remove the string before the column symbol on each column name \
Since we are only going to look at the full picture of each country in terms of gender, we will only keep the rows where the Sex column equals to Total.

In [316]:
df_new = df.pivot_table(values='OBS_VALUE:Observation Value', index=['REF_AREA:Geographic area', 'SEX:Sex', 'TIME_PERIOD:Time period'], columns='INDICATOR:Indicator', dropna=True, aggfunc='first')
df_new.reset_index(inplace=True)
df_new.columns = df_new.columns.str.split(':').str[-1].str.strip()
df_new['Geographic area'] = df_new['Geographic area'].str.split(':').str[-1].str.strip()
df_new['Sex'] = df_new['Sex'].str.split(':').str[-1].str.strip()
df_new = df_new[df_new['Sex'] == 'Total']

Check if there are any empty columns 

In [317]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3710 entries, 46 to 10731
Data columns (total 20 columns):
 #   Column                                                                                                                                                                     Non-Null Count  Dtype 
---  ------                                                                                                                                                                     --------------  ----- 
 0   Geographic area                                                                                                                                                            3710 non-null   object
 1   Sex                                                                                                                                                                        3710 non-null   object
 2   Time period                                                                               

Since column 15-19 are almost fully empty, we will remove them from the dataframe 

In [318]:
df_new = df_new.iloc[:, 0:15]

Update the data types for variables

In [319]:
## Convert time period from string to datetime
df_new['Time period'] = pd.to_datetime(df_new['Time period'], format='%Y-%m-%d').dt.year

## Convert fully numeric indicators into numeric type
def is_numeric(column):
    try:
        pd.to_numeric(column)
        return True
    except ValueError:
        return False
numeric_columns = df_new.iloc[:, 4:16].apply(is_numeric)
numeric_column_names = numeric_columns[numeric_columns].index
df_new[numeric_column_names] = df_new[numeric_column_names].apply(pd.to_numeric)
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3710 entries, 46 to 10731
Data columns (total 15 columns):
 #   Column                                                                                                                                                           Non-Null Count  Dtype  
---  ------                                                                                                                                                           --------------  -----  
 0   Geographic area                                                                                                                                                  3710 non-null   object 
 1   Sex                                                                                                                                                              3710 non-null   object 
 2   Time period                                                                                                                   

In [320]:
# Step 2: Aggregate the data for each 'Geographic area', 'Sex', and 'Time period'
# We will take the first non-null value for each group
aggregation_functions = {col: 'first' for col in df_new.columns[3:]}
df_aggregated = df_new.groupby(['Geographic area', 'Sex', 'Time period'], as_index=False).agg(aggregation_functions)

# Drop any remaining rows with all NaN values in the indicator columns
df_aggregated.dropna(subset=df_aggregated.columns[3:], how='all', inplace=True)

# Reset index to tidy up the DataFrame
df_aggregated.reset_index(drop=True, inplace=True)

df_aggregated

INDICATOR:Indicator,Geographic area,Sex,Time period,"Estimated rate of annual AIDS-related deaths (per 100,000 population)","Estimated incidence rate (new HIV infection per 1,000 uninfected population)",Reported number of children (aged 0-14 years) receiving antiretroviral treatment (ART),Per cent of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth,Reported number of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth,Estimated number of children (aged 0-17 years) who have lost one or both parents due to all causes,Estimated number of children (aged 0-17 years) who have lost one or both parents due to AIDS,Per cent of pregnant women living with HIV receiving lifelong ART,Reported number of pregnant women living with HIV receiving lifelong antiretroviral treatment (ART),Per cent of pregnant women living with HIV receiving effective ARVs for PMTCT (excludes single-dose nevirapine),Reported number of pregnant woment living with HIV receiving anitretroviral treatments (ARVs) for prevention of mother to child transmission programmes (PMTCT),Mother-to-child HIV transmission rate
0,Afghanistan,Total,2000,0.04,0.02,0.0,None,NaN,1100000.0,2000,None,NaN,None,NaN,52.70
1,Afghanistan,Total,2001,0.04,0.02,0.0,None,NaN,1120000.0,2300,None,NaN,None,NaN,52.34
2,Afghanistan,Total,2002,0.04,0.02,0.0,None,NaN,1150000.0,2700,None,NaN,None,NaN,53.31
3,Afghanistan,Total,2003,0.04,0.02,0.0,None,NaN,1170000.0,3100,None,NaN,None,NaN,52.12
4,Afghanistan,Total,2004,0.08,0.02,0.0,None,NaN,1200000.0,3600,None,NaN,None,NaN,51.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3705,Zimbabwe,Total,2018,79.94,2.23,52400.0,62.0,40000.0,960000.0,660000,92.5,59600.0,92.5,59600.0,8.15
3706,Zimbabwe,Total,2019,76.39,1.82,59800.0,57.6,35400.0,920000.0,620000,93.7,57600.0,93.7,57600.0,7.57
3707,Zimbabwe,Total,2020,73.06,1.53,56400.0,76.5,44600.0,890000.0,570000,87.9,51200.0,87.9,51200.0,7.90
3708,Zimbabwe,Total,2021,68.16,1.33,52400.0,>95,54300.0,860000.0,530000,85.4,47000.0,85.4,47000.0,8.42


In [402]:
df_aggregated.columns[[5, 7, 8, 11, 13]]

Index(['Reported number of children (aged 0-14 years) receiving antiretroviral treatment (ART)',
       'Reported number of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth',
       'Estimated number of children (aged 0-17 years) who have lost one or both parents due to all causes',
       'Reported number of pregnant women living with HIV receiving lifelong antiretroviral treatment (ART)',
       'Reported number of pregnant woment living with HIV receiving anitretroviral treatments (ARVs) for prevention of mother to child transmission programmes (PMTCT)'],
      dtype='object')

In [452]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = dbc.Container([
    dbc.Tabs([
        ### First tab
        dbc.Tab([ 
            html.H1('HIV Indicator Trends by Country and Year'),
            html.P('Select an indicator and up to 4 countries to compare their trends over time.'),
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(
                        id='indicator-dropdown',
                        value=df_aggregated.columns[3],  # Default value is the first indicator column
                        options=[{'label': col, 'value': col} for col in df_aggregated.columns[3:]], 
                        placeholder='Choose 1 indicator...'),
                    dcc.Dropdown(
                        id='country-dropdown',
                        options=[{'label': country, 'value': country} for country in df_aggregated['Geographic area'].unique()],
                        placeholder='Choose up to 4 countries...',
                        multi=True)  # Allow multiple selections
                ]),
            ]),
            dbc.Row([
                html.Iframe(id='trend-chart', style={'border-width': '0', 'width': '100%', 'height': '400px'}),
                dcc.RangeSlider(
                        id='year-slider',
                        min=df_aggregated['Time period'].min(),
                        max=df_aggregated['Time period'].max(),
                        marks={str(year): str(year) for year in range(df_aggregated['Time period'].min(), df_aggregated['Time period'].max() + 1)},
                        step=1,
                        value=[df_aggregated['Time period'].min(), df_aggregated['Time period'].max()]
                )
            ])
        ], label='Indicator Trend'),
        ### Second tab
        dbc.Tab([
            html.H1("HIV Indicator map"),
                dbc.Row([
                # Dropdown for selecting indicator
                    dcc.Dropdown(
                        id='indicator-map-dropdown',
                        # Limiting the usage of indicator columns since some of them are in non-numeric type at the moment
                        options=[{'label': col, 'value': col} for col in df_aggregated.columns[[5, 7, 8, 11, 13]]],
                        value=df_aggregated.columns[5],  # Initial value
                        multi=False,
                )]),
                dbc.Row([
                    # Map
                    dcc.Graph(id='world-map'),
                    # Slider for selecting year
                    dcc.Slider(
                        id='year-map-slider',
                        min=df_aggregated['Time period'].min(),
                        max=df_aggregated['Time period'].max(),
                        value=df_aggregated['Time period'].min(),  # Initial value
                        marks={str(year): str(year) for year in df_aggregated['Time period'].unique()},
                        step=1,
                )])],
            label='Indicator Map'    
        ),
        ### Third tab
        dbc.Tab([
            html.H1('Indicator Summary Statistics'),
            dbc.Row([
                dbc.Col([
                    dcc.Dropdown(
                        id='indicator-stats-dropdown',
                        value=df_aggregated.columns[4],  # Default value is the first indicator column
                        options=[{'label': col, 'value': col} for col in df_aggregated.columns[4:]], 
                        placeholder='Choose 1 indicator...'
                    ),
                    dcc.Dropdown(
                        id='country-stats-dropdown',
                        options=[{'label': country, 'value': country} for country in df_aggregated['Geographic area'].unique()],
                        placeholder='Choose up to 10 countries...',
                        multi=True  # Allow multiple selections, but limit to 2 for the stats comparison
                    )
                ]),
                dcc.RangeSlider(
                    id='year-stats-slider',
                    min=df_aggregated['Time period'].min(),
                    max=df_aggregated['Time period'].max(),
                    value=[df_aggregated['Time period'].min(), df_aggregated['Time period'].max()],
                    marks={str(year): str(year) for year in range(df_aggregated['Time period'].min(), df_aggregated['Time period'].max() + 1)},
                    step=1,
                    tooltip={"placement": "bottom", "always_visible": True}
                )
            ]),
            dbc.Row([
                dash_table.DataTable(id='summary-stats-table')
            ])
        ], label='Indicator Summary Statistics')
    ])    
])

# Callback for updating the chart based on selections
@app.callback(
    Output('trend-chart', 'srcDoc'),
    [Input('indicator-dropdown', 'value'),
     Input('country-dropdown', 'value'),
     Input('year-slider', 'value')]
)
def update_chart(selected_indicator, selected_countries, selected_years):
    if selected_countries is None or selected_indicator is None or len(selected_countries) > 4:
        return 'Please select an indicator and up to 4 countries.'

    # Filter based on the selected years and countries
    chart_df = df_aggregated[df_aggregated['Time period'].between(*selected_years)]
    chart_df = chart_df[chart_df['Geographic area'].isin(selected_countries)]
    
    # Create the Altair chart
    base = alt.Chart(chart_df).encode(
        x=alt.X('Time period:O', axis=alt.Axis(title='Year')),
        y=alt.Y(f"{selected_indicator}:Q", axis=alt.Axis(title=selected_indicator)),
        color='Geographic area:N'
    )
    
    line_chart = base.mark_line(point=True).properties(
        width=700,
        height=400
    )
    
    return line_chart.to_html()

# Callback for updating the map based on dropdown and slider values
@app.callback(
    Output('world-map', 'figure'),
    [Input('indicator-map-dropdown', 'value'),
     Input('year-map-slider', 'value')]
)
def update_map(selected_indicator, selected_year):
    filtered_data = df_aggregated[(df_aggregated['Time period'] == selected_year)]
    fig = px.scatter_geo(
        filtered_data,
        locations='Geographic area',
        locationmode='country names',
        color=selected_indicator,
        hover_name='Geographic area',
        # Limit the information to display on the hover box
        hover_data={'Geographic area':False ,
                    'Reported number of children (aged 0-14 years) receiving antiretroviral treatment (ART)':False,
                    'Reported number of infants born to pregnant women living with HIV who received a virological test for HIV within 2 months of birth':False,
                    'Estimated number of children (aged 0-17 years) who have lost one or both parents due to all causes':False,
                    'Reported number of pregnant women living with HIV receiving lifelong antiretroviral treatment (ART)':False,
                    'Reported number of pregnant woment living with HIV receiving anitretroviral treatments (ARVs) for prevention of mother to child transmission programmes (PMTCT)':False},
        labels={'size': 'Value'},
        size=filtered_data[selected_indicator].fillna(0),
        size_max=35,
        projection='natural earth',
    )
    # Remove the legend since some of the column names are too long and will squeeze the size of the map
    fig.update(layout_coloraxis_showscale=False)
    fig.update_geos(
        showcountries=True, countrycolor='whitesmoke',
        showland=True, landcolor='dimgrey',
        showocean=True, oceancolor="Black"
    )
    fig.update_layout(
        title=f"{selected_indicator} in {selected_year}",
        title_x=0.5,
        title_font_size=14
    )
    return fig

# Callback for updating the summary statistics table
@app.callback(
    [Output('summary-stats-table', 'data'), Output('summary-stats-table', 'columns')],
    [Input('indicator-stats-dropdown', 'value'),
     Input('country-stats-dropdown', 'value'),
     Input('year-stats-slider', 'value')]
)
def update_summary_statistics(selected_indicator, selected_countries, selected_years):
    print("Callback Triggered")  # For debugging, check the console where you run the server

    if not selected_countries or not selected_indicator or len(selected_countries) > 10:
        return [], []

    # Filter based on the selected years and countries
    stats_df = df_aggregated[df_aggregated['Time period'].between(*selected_years)]
    stats_df = stats_df[stats_df['Geographic area'].isin(selected_countries)]

    # Compute summary statistics
    summary = stats_df.groupby('Geographic area')[selected_indicator].agg(['mean', 'min', 'max', 'count']).reset_index()

    # Rename columns for better readability in the DataTable
    summary.columns = ['Geographic area', 'Mean', 'Min', 'Max', 'Non-null Count']

    # Convert to records format for DataTable
    data = summary.to_dict('records')
    columns = [{"name": i, "id": i} for i in summary.columns]

    return data, columns

if __name__ == '__main__':
    app.run_server(debug=True)

Callback Triggered
Callback Triggered
Callback Triggered
---------------------------------------------------------------------------
NotImplementedError                       Traceback (most recent call last)
File ~/miniconda3/lib/python3.12/site-packages/pandas/core/groupby/groupby.py:1791, in GroupBy._cython_agg_general.<locals>.array_func(
    values=array(['0.01', '0.01', '0.02', '0.02', '0.02', '...03', '0.03', '0.03', '0.03'],
      dtype=object)
)
   1790 try:
-> 1791     result = self.grouper._cython_operation(
        self.grouper = <pandas.core.groupby.ops.BaseGrouper object at 0x147e5b5f0>
        self = <pandas.core.groupby.generic.SeriesGroupBy object at 0x150d11af0>
        values = array(['0.01', '0.01', '0.02', '0.02', '0.02', '0.02', '0.02', '0.02',
       '0.02', '0.03', '0.03', '0.03', '0.03', '0.03', '0.03', '0.03',
       '0.03', '0.03', '0.03', '0.03', '0.03', '0.03', '0.03'],
      dtype=object)
        how = 'mean'
        data = SingleBlockManager
Items: Int64I